In [1]:
import pandas as pd
import networkx as nx
from node2vec import Node2Vec
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from sklearn.preprocessing import StandardScaler
import numpy as np
from rdkit.Chem import rdmolops
from tqdm import tqdm
from IPython.display import clear_output
import seaborn as sns 
import matplotlib.pyplot as plt
import os

In [2]:
tox21 = pd.read_csv('data/tox21.csv')
smiles_list = tox21['SMILES'].tolist()
label_columns = ['SR-HSE','NR-AR', 'SR-ARE', 'NR-Aromatase', 'NR-ER-LBD', 'NR-AhR', 'SR-MMP',\
       'NR-ER', 'NR-PPAR-gamma', 'SR-p53', 'SR-ATAD5', 'NR-AR-LBD']
targets = tox21[label_columns]


In [3]:
# Functions for Feature Extraction
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    descriptors = {
        "MolecularWeight": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "HBD": Descriptors.NumHDonors(mol),
        "HBA": Descriptors.NumHAcceptors(mol),
        "TPSA": Descriptors.TPSA(mol),
    }
    return descriptors

def compute_fingerprints(smiles):
    mol = Chem.MolFromSmiles(smiles)
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
    return {
        "MorganFP": list(morgan_fp),
        "MACCSFP": list(maccs_fp),
    }

def compute_graph_features(graph):
    features = {
        "AverageDegree": sum(dict(graph.degree()).values()) / len(graph.nodes()),
        "Density": nx.density(graph),
        "ClusteringCoefficient": nx.average_clustering(graph),
    }
    if nx.is_connected(graph):
        features["Diameter"] = nx.diameter(graph)
    else:
        features["Diameter"] = None
    return features

def compute_node2vec_embeddings(graph, dimensions=64):
    node2vec = Node2Vec(graph, dimensions=dimensions, walk_length=30, num_walks=50, workers=12, seed=42)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    
    node_embeddings = np.array([model.wv[str(node)] for node in graph.nodes()])
    graph_embedding = node_embeddings.mean(axis=0)  
    return graph_embedding

def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    adjacency_matrix = rdmolops.GetAdjacencyMatrix(mol)
    graph = nx.from_numpy_array(adjacency_matrix)
    for i, atom in enumerate(mol.GetAtoms()):
        graph.nodes[i]['atom_type'] = atom.GetSymbol()
    return graph

In [4]:
if not os.path.exists('data/features.csv'):
    features = []
    for smile in tqdm(smiles_list):
        try:
            mol = Chem.MolFromSmiles(smile)
            graph = smiles_to_graph(smile)
            
            descriptors = compute_descriptors(smile)
            fingerprints = compute_fingerprints(smile)
            graph_features = compute_graph_features(graph)
            node2vec_embeddings = compute_node2vec_embeddings(graph)
            
            combined_features = {**descriptors, **fingerprints, **graph_features, **{f"Node2Vec{i}": node2vec_embeddings[i] for i in range(len(node2vec_embeddings))}}
            features.append(combined_features)
        except Exception as e:
            print(f"Tus muertos pisados {smile}: {e}")
        clear_output(wait=True)

    feature_df = pd.DataFrame(features)
    feature_df.to_csv('data/features.csv', index=False)
else:
    feature_df = pd.read_csv('data/features.csv')


In [5]:
#feature_df.drop(
#    ['MorganFP', 'MACCSFP'], axis=1, inplace=True
#)


In [6]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

morgan = feature_df['MorganFP']
maccs = feature_df['MACCSFP']
feature_df.drop(['MorganFP', 'MACCSFP'], axis=1, inplace=True)

iterative_imputer = IterativeImputer(max_iter=10, random_state=42)
targets = pd.DataFrame(iterative_imputer.fit_transform(targets), columns=targets.columns)
feature_df = pd.DataFrame(iterative_imputer.fit_transform(feature_df), columns=feature_df.columns)


C:\Users\vmarc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\impute\_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [7]:
targets = targets.map(lambda x: 1 if x > 0.5 else 0)
targets

,SR-HSE,NR-AR,SR-ARE,NR-Aromatase,NR-ER-LBD,NR-AhR,SR-MMP,NR-ER,NR-PPAR-gamma,SR-p53,SR-ATAD5,NR-AR-LBD
0,0,0,1,1,0,1,1,1,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,1,0,1,1,0,1,1,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8034,0,0,0,0,0,0,0,0,0,0,0,0
8035,0,0,0,0,0,0,0,0,0,0,0,0
8036,0,0,0,0,0,0,0,0,0,0,0,0
8037,0,0,1,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer, f1_score, classification_report
from sklearn.decomposition import PCA

model = MultiOutputClassifier(XGBClassifier(n_estimators=1000, max_depth=6, learning_rate=0.1, n_jobs=-1, random_state=42, scale_pos_weight=10, enable_categorical = True, eval_metric = 'logloss'), n_jobs=-1)

X,y = feature_df, targets.astype(int)


weights = {}
for column in y.columns:
    class_counts = y[column].value_counts()
    weights[column] = class_counts[0] / class_counts[1] if 1 in class_counts else 1
    

classifiers = []

for column in y.columns:
    
    xgb = XGBClassifier(
        use_label_encoder=False,  
        n_jobs=-1,
        random_state=42,
        eval_metric='logloss',
        scale_pos_weight=weights[column],
        enable_categorical = True
    )
    
    classifiers.append(xgb)

model = MultiOutputClassifier(estimator = XGBClassifier(enable_categorical = True), n_jobs=-1)
model.estimators_ = classifiers


#pca = PCA(n_components=2)
#X_train_2d = pca.fit_transform(X_train)
#model.fit(X_train, y_train)




In [9]:
targets[targets['NR-AR'] == 1]

,SR-HSE,NR-AR,SR-ARE,NR-Aromatase,NR-ER-LBD,NR-AhR,SR-MMP,NR-ER,NR-PPAR-gamma,SR-p53,SR-ATAD5,NR-AR-LBD
22,1,1,1,1,1,0,0,1,0,0,0,1
39,0,1,1,1,0,0,1,0,0,1,0,1
64,0,1,0,0,0,0,0,0,0,1,0,0
92,1,1,1,1,1,0,1,1,0,1,0,1
100,0,1,1,0,1,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6233,0,1,0,0,1,0,0,1,0,0,0,0
6282,0,1,1,0,1,0,0,1,0,0,0,1
6709,0,1,1,0,0,0,0,0,0,0,0,0
7009,0,1,0,0,0,0,0,0,0,0,0,0


In [10]:
import pickle
model.fit(X, y)
with open('models/model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [11]:
aaaa

NameError: name 'aaaa' is not defined

In [ ]:
y_pred = model.predict(X_test)

f1_scores = []
for i, column in enumerate(y.columns):
    f1 = f1_score(y_test.iloc[:, i], y_pred[:, i], average="macro", zero_division=0)
    f1_scores.append(f1)
    print(f"F1-Score for {column}: {f1:.4f}")
    
mean_f1 = sum(f1_scores) / len(f1_scores)
print(f"\nMean F1-Score across all outputs: {mean_f1:.4f}")

for i, column in enumerate(y.columns):
    print(f"\nClassification Report for {column}:\n")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

mat = multilabel_confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(3, 4, figsize=(15, 10))
axes = axes.ravel()

for i, (ax, label) in enumerate(zip(axes, label_columns)):
    sns.heatmap(mat[i], annot=True, fmt='d', ax=ax, cmap='coolwarm', cbar=False)
    ax.set_title(label)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')

plt.tight_layout()
plt.show()
